# Practica 5
***
Rodriguez Nuñez Diego Eduardo

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("NaiveBayesSentimientos") \
    .getOrCreate()


In [3]:
df = spark.read.csv("sentimientos.csv", header=True, inferSchema=True)
df.show(5)


+--------------------+-----------+
|               texto|sentimiento|
+--------------------+-----------+
|Me encanta este p...|   positivo|
|Este producto es ...|   negativo|
|La experiencia fu...|   positivo|
|No me gusta el se...|   negativo|
|Definitivamente l...|   positivo|
+--------------------+-----------+
only showing top 5 rows



In [7]:
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="texto", outputCol="palabras")
df_tokenized = tokenizer.transform(df)

df.printSchema()
df.show(5)



root
 |-- texto: string (nullable = true)
 |-- sentimiento: string (nullable = true)

+--------------------+-----------+
|               texto|sentimiento|
+--------------------+-----------+
|Me encanta este p...|   positivo|
|Este producto es ...|   negativo|
|La experiencia fu...|   positivo|
|No me gusta el se...|   negativo|
|Definitivamente l...|   positivo|
+--------------------+-----------+
only showing top 5 rows



In [9]:
from pyspark.ml.feature import CountVectorizer

vectorizer = CountVectorizer(inputCol="palabras", outputCol="features")
vectorizer_model = vectorizer.fit(df_tokenized)
df_vectores = vectorizer_model.transform(df_tokenized)


In [10]:
from pyspark.sql.functions import when

df_final = df_vectores.withColumn("label", when(df_vectores.sentimiento == "positivo", 1).otherwise(0))


In [11]:
datos_entrenamiento, datos_prueba = df_final.randomSplit([0.8, 0.2], seed=42)


In [12]:
from pyspark.ml.classification import NaiveBayes

modelo = NaiveBayes(featuresCol="features", labelCol="label", modelType="multinomial")
modelo_entrenado = modelo.fit(datos_entrenamiento)


In [14]:
predicciones = modelo_entrenado.transform(datos_prueba)
predicciones.select("texto", "label", "prediction").show(10)



+--------------------+-----+----------+
|               texto|label|prediction|
+--------------------+-----+----------+
|El color no es co...|    0|       0.0|
|El vendedor no re...|    0|       0.0|
|Este producto no ...|    0|       0.0|
|La experiencia fu...|    1|       0.0|
|Me siento feliz c...|    1|       1.0|
|No me gusta el se...|    0|       0.0|
|lo volveri�a a co...|    1|       1.0|
+--------------------+-----+----------+



In [15]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluador = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
precision = evaluador.evaluate(predicciones)

print(f"Precisión del modelo: {precision * 100:.2f}%")


Precisión del modelo: 85.71%
